In [1]:
def hex2bin(s):
	mp = {'0': "0000",
		'1': "0001",
		'2': "0010",
		'3': "0011",
		'4': "0100",
		'5': "0101",
		'6': "0110",
		'7': "0111",
		'8': "1000",
		'9': "1001",
		'A': "1010",
		'B': "1011",
		'C': "1100",
		'D': "1101",
		'E': "1110",
		'F': "1111"}
	bin = ""
	for i in range(len(s)):
		bin = bin + mp[s[i]]
	return bin


In [2]:
def bin2hex(s):
	mp = {"0000": '0',
		"0001": '1',
		"0010": '2',
		"0011": '3',
		"0100": '4',
		"0101": '5',
		"0110": '6',
		"0111": '7',
		"1000": '8',
		"1001": '9',
		"1010": 'A',
		"1011": 'B',
		"1100": 'C',
		"1101": 'D',
		"1110": 'E',
		"1111": 'F'}
	hex = ""
	for i in range(0, len(s), 4):
		# ch = ""
		# ch = ch + s[i]
		# ch = ch + s[i + 1]
		# ch = ch + s[i + 2]
		# ch = ch + s[i + 3]
		ch = s[i:i+4]
		hex = hex + mp[ch]

	return hex


In [3]:
def bin2dec(s):
    dec = 0
    sle = len(s)
    for i in range(sle):
        dec+=int(s[i])*(2**(sle-i-1))
    return dec

In [4]:
def dec2bin(n,length):
    bin = ''
    m = n
    while m!=0:
        rem = m%2
        m//=2
        bin = str(rem) + bin
    l = len(bin)
    # if l<length and l!=-1:
    padding = (length-l)*'0'
    bin =  padding + bin
    return bin

In [5]:
dec2bin(3,4)

'0011'

## Key generation


We start with a 64 bit key

In [6]:
key_arr = list('0111101100101101001001010111011001110101001101010111100100110111')

In [7]:
def parity_drop(key_arr):
    out_arr = []
    order_arr = [57, 49, 41, 33, 25, 17, 9,
                1, 58, 50, 42, 34, 26, 18,
                10,  2, 59, 51, 43, 35, 27,
                19, 11,  3, 60, 52, 44, 36,
                63, 55, 47, 39, 31, 23, 15,
                7, 62, 54, 46, 38, 30, 22,
                14,  6, 61, 53, 45, 37, 29,
                21, 13,  5, 28, 20, 12,  4]
    for i in range(len(order_arr)):
        out_arr.append(key_arr[order_arr[i]-1])
    return out_arr


In [8]:
pd_key_arr = parity_drop(key_arr)
len(pd_key_arr)

56

In [9]:
def PC2(key_arr):
    out_arr = []
    order_arr = [14, 17, 11, 24,  1,  5,
                3, 28, 15,  6, 21, 10,
                23, 19, 12,  4, 26,  8,
                16,  7, 27, 20, 13,  2,
                41, 52, 31, 37, 47, 55,
                30, 40, 51, 45, 33, 48,
                44, 49, 39, 56, 34, 53,
                46, 42, 50, 36, 29, 32]
    for i in range(len(order_arr)):
        out_arr.append(key_arr[order_arr[i]-1])
    return out_arr


In [10]:
def round_key_generator(pd_key_arr):
    '''Generates the keys for each round of DES
    Input:
    The key of DES after parity drop
    '''
    
    round_keys = []

    # Splitting into left and right halves
    left = pd_key_arr[:28]
    right = pd_key_arr[28:]


    for i in range(16):
        # Shifting both halves left by one place circularly
        left = left[1:] + [left[0]]
        right = right[1:] + [right[0]]

        round_keys.append(PC2(left+right))
    
    return round_keys

In [11]:
round_keys = round_key_generator(pd_key_arr)
len(round_keys[0])

48

## Encryption

### Step 1: Initial permutation

In [12]:
def initial_permutation(input_arr:str):
    output_arr = []
    ordering_arr = [58, 50, 42, 34, 26, 18, 10, 2,
                    60, 52, 44, 36, 28, 20, 12, 4,
                    62, 54, 46, 38, 30, 22, 14, 6,
                    64, 56, 48, 40, 32, 24, 16, 8,
                    57, 49, 41, 33, 25, 17, 9, 1,
                    59, 51, 43, 35, 27, 19, 11, 3,
                    61, 53, 45, 37, 29, 21, 13, 5,
                    63, 55, 47, 39, 31, 23, 15, 7]
    for i in range(64):
        output_arr.append(input_arr[ordering_arr[i]-1])
    return output_arr

In [13]:
inp_arr = list('0110111001101001011000110110011101100111011011110110111101100001')
curr_arr = initial_permutation(inp_arr)

### Step 2: DES function

We split the input(**64 bits**) into 2 parts of **32 bits** each

In [14]:
lt_round_0_init = curr_arr[:32]
rt_round_0_init = curr_arr[32:]

lt_round_0 = lt_round_0_init
rt_round_0 = rt_round_0_init

#### Step 2.1 Expansion permutation
input : 32 bits  
output : 64 bits

In [15]:
def ExpansionPBlock(input_arr: str):
    output_arr = []
    ordering_arr = [32,  1,  2,  3,  4,  5,
                    4,  5,  6,  7,  8,  9,
                    8,  9, 10, 11, 12, 13,
                    12, 13, 14, 15, 16, 17,
                    16, 17, 18, 19, 20, 21,
                    20, 21, 22, 23, 24, 25,
                    24, 25, 26, 27, 28, 29,
                    28, 29, 30, 31, 32,  1]
    for i in range(48):
        output_arr.append(input_arr[ordering_arr[i]-1])
    return output_arr    

In [16]:
rt_round_0 = ExpansionPBlock(rt_round_0)

#### Step 2.2 Whitener(XOR)
We perform a XOR operation of the outcome and the round key (Only performing round 1 here)

In [17]:
def whitener(rt_round,round_keys,round_num):
    # converting list to str to int for XOR operation
    rt_round_str = ''.join(rt_round)
    # print(len(rt_round_str))
    rt_round_int = bin2dec(rt_round_str)
    # print(rt_round_int)

    round_key = round_keys[round_num]
    round_key_str = ''.join(round_key)
    # print(len(round_key_str))
    round_key_int = bin2dec(round_key_str)
    # print(round_key_int)

    rt_round_int = rt_round_int ^ round_key_int
    # print(rt_round_int)
    rt_round_str = dec2bin(rt_round_int,48)
    print(rt_round_str)
    rt_round = list(rt_round_str)

    return rt_round

In [18]:
rt_round_0 = whitener(rt_round_0,round_keys,0)

111100001011100101010010011101011010100011100111


In [19]:
# # converting list to str to int for XOR operation
# rt_round_0_str = ''.join(rt_round_0)
# print(len(rt_round_0_str))
# rt_round_0_int = bin2dec(rt_round_0_str)
# print(rt_round_0_int)

# round_key_0 = round_keys[0]
# round_key_0_str = ''.join(round_key_0)
# print(len(round_key_0_str))
# round_key_0_int = bin2dec(round_key_0_str)
# print(round_key_0_int)

# rt_round_0_int = rt_round_0_int ^ round_key_0_int
# print(rt_round_0_int)
# rt_round_0_str = dec2bin(rt_round_0_int,48)
# print(rt_round_0_str)
# rt_round_0 = list(rt_round_0_str)

#### Step 2.3 S boxes

In [20]:
# a = list('1001')
# row = a[0] + a[-1]
# col = a[1:-1]
# row = bin2dec(''.join(row))
# row

In [21]:
Sboxes_order = [
    [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    [
    [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
    [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
    [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
    [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    [
    [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
    [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
    [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
    [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    [
    [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
    [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
    [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
    [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    [
    [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
    [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
    [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
    [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    [
    [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
    [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
    [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
    [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    [
    [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
    [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
    [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
    [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    [
    [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
    [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
    [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
    [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

In [22]:
def sbox_val(inp_arr,sbox):

    # Getting row and col values from the input
    row = inp_arr[0] + inp_arr[-1]
    row_int = bin2dec(''.join(row))
    col = inp_arr[1:-1]
    col_int = bin2dec(''.join(col))

    # finding the substition value
    out_int = sbox[row_int][col_int]
    print(f'{out_int=}')
    out_str = dec2bin(out_int,4)
    print(out_str)
    return list(out_str)

    

In [23]:
def Sboxes(rt_round,sboxes):
    # First we divide our current output into 8 parts(6 bits each)
    s_inps = [rt_round_0[:6],
        rt_round_0[6:12],
        rt_round_0[12:18],
        rt_round_0[18:24],
        rt_round_0[24:30],
        rt_round_0[30:36],
        rt_round_0[36:42],
        rt_round_0[42:48]
    ]
    
    sbox_outs = []
    # getting the values for S box
    for i in range(8):
        sbox_outs += sbox_val(s_inps[i],Sboxes_order[i])
    
    return sbox_outs

In [24]:
rt_round_0 = Sboxes(rt_round_0,Sboxes_order)

out_int=5
0101
out_int=2
0010
out_int=13
1101
out_int=2
0010
out_int=8
1000
out_int=7
0111
out_int=11
1011
out_int=7
0111


In [25]:
''.join(rt_round_0)

'01010010110100101000011110110111'

#### Step 2.4 Straight permutation box

In [26]:
def straight_pbox(inp_arr):
    out_arr = []
    order_arr = [16,  7, 20, 21,
                29, 12, 28, 17,
                1, 15, 23, 26,
                5, 18, 31, 10,
                2,  8, 24, 14,
                32, 27,  3,  9,
                19, 13, 30,  6,
                22, 11,  4, 25]
    for i in range(32):
        out_arr.append(inp_arr[order_arr[i]-1])
    return out_arr

In [27]:
rt_round_0 = straight_pbox(rt_round_0)

#### Step 2.5 XOR with left side

In [28]:
# Converting to int first
lt_round_0_str = ''.join(lt_round_0)
rt_round_0_str = ''.join(rt_round_0)
print(f'{lt_round_0_str=}\n{rt_round_0_str=}')

# Converting to decimal and XOR
lt_round_0_int = bin2dec(lt_round_0_str)
rt_round_0_int = bin2dec(rt_round_0_str)
print(f'{lt_round_0_int=}\n{rt_round_0_int=}')
lt_round_0_final_int = lt_round_0_int ^ rt_round_0_int
print(lt_round_0_final_int)

# Converting back to a list
lt_round_0_final_str = dec2bin(lt_round_0_final_int,32)
lt_round_0_final = list(lt_round_0_final_str)
print(lt_round_0_final)


lt_round_0_str='11111111000000000111100111111110'
rt_round_0_str='01000111011000111010110100101011'
lt_round_0_int=4278221310
rt_round_0_int=1197714731
3093550293
['1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1']


In [29]:
rt_round_0_final = rt_round_0_init

### Step 3: Swap
We will swap the left and right half of the cipher

In [30]:
lt_round_1_init = rt_round_0_final
rt_round_1_init = lt_round_0_final

So, the begining value of round 1 will look something as follows

In [31]:
''.join(lt_round_1_init+rt_round_1_init)

'0000000011111111011000110111110110111000011000111101010011010101'